In [ ]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/expanse/lustre/projects/csd819/a3murali/cache/'

from huggingface_hub import login
login(token = "<hf_token>")

import transformers
import torch

import csv
import numpy as np
import pandas as pd

print(f"PyTorch Version: {torch.__version__}")
print(f"Transformers Version: {transformers.__version__}")

from transformers import AutoConfig, AutoModel, AutoTokenizer, AutoModelForCausalLM, pipeline, LlamaForCausalLM

model = "ivnle/llamatales_jr_8b-lay8-hs512-hd8-33M"
tokenizer = AutoTokenizer.from_pretrained(model)


llamatales_pipeline = pipeline("text-generation", model = model, device = "cuda")

prompts = {
    1: "Once upon a time there was a dragon", 
    2: "Once upon a time there was a princess", 
    3: "Once upon a time there were two children",
    4: "Once upon a time there was a prince",
    5: "Once upon a time there was a frog",
    6: "Once upon a time there was a king",
    7: "Once upon a time there was a queen",
    8: "Once upon a time there was a wolf",
    9: "Once upon a time there was a genie",
    10: "Once upon a time there was a poor boy"
}

data = []
for prompt_id in prompts:
    #generation - generate stories for each prompt
    for i in range(1):
        sequences = llamatales_pipeline(
            prompts[prompt_id],
            do_sample = True,
            top_k = 10,
            num_return_sequences = 1,
            max_new_tokens = 512
        )
        generated_story = sequences[0]['generated_text']
        print(generated_story)

        num_tokens_generated_story = len(tokenizer.encode(generated_story))

        #test - collect hidden states
        
        config = AutoConfig.from_pretrained("/expanse/lustre/projects/csd819/a3murali/cache/models--ivnle--llamatales_jr_8b-lay8-hs512-hd8-33M/snapshots/461f50f0024efb46b94dc68cc850d12d75ecb325", output_hidden_states = True)
        model = AutoModelForCausalLM.from_config(config).to('cuda')
        #print(model)

#         for i in model.named_parameters():
#             print(f"{i[0]} -> {i[1].device}")


        tokenizer = AutoTokenizer.from_pretrained("/expanse/lustre/projects/csd819/a3murali/cache/models--ivnle--llamatales_jr_8b-lay8-hs512-hd8-33M/snapshots/461f50f0024efb46b94dc68cc850d12d75ecb325/", config = config)

        inputs = tokenizer(generated_story, return_tensors="pt").to("cuda")

        outputs = model.generate(inputs.input_ids, attention_mask=inputs["attention_mask"], do_sample = True, top_k = 10, num_return_sequences = 1, max_new_tokens = 512, eos_token_id = tokenizer.eos_token_id, pad_token_id = tokenizer.pad_token_id, return_dict_in_generate=True)

        #analysis - collect prompt id of each token
        output_id = []
        for token in range(num_tokens_generated_story):
            output_id.append(prompt_id)
        
        convert_hidden_states = []

        for i in outputs.hidden_states:
            convert_hidden_states.append([j.detach().cpu().numpy() for j in i])
        
        print(len(convert_hidden_states))
        print(len(convert_hidden_states[0]))
        print(len(convert_hidden_states[0][0]))
        print(len(convert_hidden_states[0][0][0]))
        print(len(convert_hidden_states[0][0][0][0]))
        #print(len(convert_hidden_states[0][0][0][0][0]))
        data.append([prompts[prompt_id], generated_story, convert_hidden_states, np.array(output_id)])

        #(([]))

df = pd.DataFrame(data, columns = ["prompt", "story", "hidden_states", "output_token_prompt_id"], dtype = 'object')
df

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/a3murali/.cache/huggingface/token
Login successful
PyTorch Version: 1.13.0a0+d321be6
Transformers Version: 4.35.0


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Once upon a time there was a dragon named Scorch. Scorch was a very friendly dragon. He lived in a cave with his best friend, a little rabbit named Lily. One day, Lily and Scorch were playing outside when a big storm came. The wind was blowing hard, and the thunder was loud.

Scorch wanted to help Lily. So he breathed a big fire to make the storm go away. But the fire was so big that it started to burn Lily's house! Scorch thought, "I must do something." He remembered that he could use his fire to warm up the fireplace. He roared loudly to get a grown-up who was nearby. The grown-up quickly got some water and put out the fire.

Lily was so happy that her house was safe again. The storm went away, and the sun came out. Lily hugged Scorch and said, "Thank you, dear Scorch. You are the best dragon ever!" Scorch smiled, and they both played in the sunshine. From that day on, Lily learned that even when things seem bad, we can make them better with our help from our friends.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


512
9
1
233
512
Once upon a time there was a princess. She lived in a big castle with a moat all around it. One day, she got a big, bad idea. She wanted to make the moat a big mud castle.

She picked flowers and rocks and started to build the mud castle. She thought it would be fun. But then, she met a friendly fish who said, "Hello, princess! I can help you build a castle."

The princess trusted the fish, so she gave him a big bucket of water. The fish swam up to the bucket and started to dig in the mud. But, he knocked the bucket over, and all the water spilled out! The princess was very sad. She said, "Oh no! Now the water is gone, and I don't have enough!"

The fish said, "It's okay, princess. I didn't mean to spill the water. But you should have been more careful." The princess felt a little better, but she was still sad. She realized she should have been more careful with the water.

The princess was left with mud and a broken bucket, a big problem. She learned that sometimes, no

...

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


512
9
1
268
512
Once upon a time there was a poor boy named Timmy. He had a very small dog named Max. They lived on a big ranch where they could run and play every day.

One sunny day, while Timmy was playing, a strong wind came and blew Max's hat off his head. Max was very sad because he loved his red hat and was trying to get it back. Timmy tried to help him, but the wind was blowing too hard for him to hold onto the hat. Then, a friendly horse came by and gently took the hat from Max.

But, to Timmy's surprise, the friendly horse was actually a magician! He was making things disappear and reappear on the hat! Timmy gasped in amazement. The magician said, "This hat was special. It can make anyone or anything happen when it rains or blows." Timmy was so happy to see this. The magician gave Max a special prize, a special collar with a bell on it, so Max could wear the hat whenever he wanted.

But that was not all. When Timmy and Max went home, the sun was still shining, and everything 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


512
9
1
271
512


,prompt,story,hidden_states,output_token_prompt_id
0,Once upon a time there was a dragon,Once upon a time there was a dragon named Scor...,[[[[[-0.00981658 -0.01614933 -0.0005933 ... -...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,Once upon a time there was a princess,Once upon a time there was a princess. She liv...,[[[[[-0.03631564 -0.00093168 0.00519177 ... -...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
2,Once upon a time there were two children,Once upon a time there were two children named...,[[[[[ 0.02472341 -0.01180643 -0.00662811 ... ...,"[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
3,Once upon a time there was a prince,Once upon a time there was a prince. He lived ...,[[[[[-4.4862102e-03 2.6884653e-02 -1.3762116e...,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ..."
4,Once upon a time there was a frog,Once upon a time there was a frog named Freddy...,[[[[[-0.02513336 -0.00119402 0.00494934 ... ...,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
5,Once upon a time there was a king,Once upon a time there was a king. The king lo...,[[[[[-4.24982794e-03 1.36922384e-02 1.602065...,"[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ..."
6,Once upon a time there was a queen,Once upon a time there was a queen named Queen...,[[[[[ 0.01051055 0.00664904 -0.00644722 ... ...,"[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ..."
7,Once upon a time there was a wolf,Once upon a time there was a wolf named Rufus....,[[[[[ 0.01185705 0.00195181 -0.02484797 ... ...,"[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ..."
8,Once upon a time there was a genie,Once upon a time there was a genie who lived i...,[[[[[ 0.00185348 0.01174646 -0.02049438 ... -...,"[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ..."
9,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,[[[[[ 0.0211226 0.00415412 0.01156262 ... ...,"[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1..."
